In [2]:
import os 
from os import path 
import shutil
import csv
from PIL import Image
import numpy as np
import tabulate 

#Step-1
def create_directory(path,directory_name):

    directory_dest = path + "/" + directory_name
    os.mkdir(directory_dest)
    
def create_dictionary_files(list_files,resource):
    dictionary_files={}
    for name in list_files:
        if name == "recap.csv":
            continue
        
        resource_file = resource + "/" + name
        if path.isfile(resource_file) :
            size = path.getsize(resource_file)
            name_split = path.splitext(name)
            # Se dovessimo prendere in considerazione piu tipi diversi di estensione 
            # basterebbe creare tre insiemi (images,docs,audio) di tipo set() nei quali bisognerebbe 
            # inizializzare tutte le estensioni che vogliamo prendere in considerazione per ogni tipologia di file.
            if name_split[1] == ".jpg" or name_split[1] == ".png" or name_split[1] == ".jpeg":
                dictionary_files[name]= {"type" : "images","size":size}
            elif name_split[1] == ".txt" or name_split[1] == ".odt" or name_split[1] == ".docx":
                dictionary_files[name]= {"type" : "docs","size":size}
            else:
                dictionary_files[name]= {"type" : "audio","size":size}

    return dictionary_files

def create_set_types(path_files):
    exist = False
    set_types = set()
    name_elements = os.listdir(path_files)
    name_files = list(name_elements)
    for name in name_elements: 
        if path.isdir(path_files + "/" + name):
            name_files.remove(name)
            set_types.add(name)
        elif name == "recap.csv":
            exist = True 

    return set_types,name_files,exist

def iteration_files(position_files):
    # Per verificare se la directory esiste già, creo un insieme con le tre categorie (immagini, documenti, audio).
    set_types,name_files, exist = create_set_types(position_files)
    # Creo un dizionario per abbinare ogni file al suo tipo (immagine, documento, audio).
    dictionary_files = create_dictionary_files(name_files,position_files)
    if dictionary_files == None :
        exit("There are no free files on the macro directory")
    #Iterazione nel file
    for name,info in dictionary_files.items():
        print(f"{name} type:{info['type']} size:{info['size']}\n")
        # Controllare se la cartella esiste già
        # Se non esiste, aggiungere la directory e spostare i file nella directory
        if info["type"] not in set_types:
            set_types.add(info["type"])
            create_directory(position_files,info["type"])

            # spostare i file nella directory giusta 
            resource = position_files + "/" + name
            dest_path = position_files + "/" + info["type"]
            shutil.move(resource,dest_path)

        else:
            # spostare i file nella directory giusta
            resource = position_files + "/" + name
            dest_path = position_files + "/" + info["type"]
            shutil.move(resource,dest_path)
    
    return dictionary_files, exist

def recap(path_position,dictionary,exist):

    name_path = path_position + "/" + "recap.csv"
    if not exist :
        with open(name_path,'w', encoding="UTF8", newline="") as f:
            writer = csv.writer(f)
            # Header
            writer.writerow(["name","type","size(B)"])
    
    with open(name_path,"a", encoding="UTF8", newline="") as f:
        writer = csv.writer(f)
        for name,info in dictionary.items():
            name_clean = path.splitext(name)[0]
            #Files  
            writer.writerow([name_clean, info["type"], str(info["size"])])

#Step-3
def create_array_informations(name,directory):

    resource = directory + "/" + name
    im = Image.open(resource)
    a = np.array(im)
    return a

def create_list_info(list,direct_images):
    # Ho scelto di utilizzare un elenco perché era meglio stampare con il modulo "tabulate". 
    list_info = []
    # Iterazione nella directory delle immagini
    for name in list:
        name_clean = os.path.splitext(name)[0]
        x = create_array_informations(name,direct_images)
        tipology = np.shape(x)

        # Trovare i valori medi (scala di grigi, R,G,B,ALPHA) per ogni immagine
        if len(tipology) == 2:
            grayscale = round(np.mean(x),2)
            # Inserisco all'interno della tabella i valori che ci interessano 
            list_info.append([name_clean,tipology[0],tipology[1],grayscale,0.00,0.00,0.00,0.00])
        else:
            alpha = 0 
            tupla_averages = np.mean(x,axis=(0,1))
            if tipology[2] == 4 : 
                alpha = round(tupla_averages[3],2)
            R = round(tupla_averages[0],2)
            G = round(tupla_averages[1],2)
            B = round(tupla_averages[2],2)
            # Inserisco all'interno della tabella i valori che ci interessano
            list_info.append([name_clean,tipology[0],tipology[1],0.00,R,G,B,alpha])
      
    return list_info

def printa_tabule(info):
    headers = ["name","height","width","greyscale","R","G","B","ALPHA"]
    print(tabulate.tabulate(info,headers,tablefmt="grid"))

def main():
     
    position_files= "" ## INSERIRE Path della Posizione della directory "file" 
    informations_of_files, exist = iteration_files(position_files)
    recap(position_files,informations_of_files, exist)

    directory_images = position_files + "/images"
    list_of_images = os.listdir(directory_images)
    # Creazione di un database tramite tabella con le informazioni che ci servono per ogni immagine 
    list_info = create_list_info(list_of_images,directory_images)
    # Print della tabella come richiesto dal testo
    printa_tabule( list_info ) 

if __name__ == "__main__":
    main()


WhatsApp Image 2022-09-28 at 19.07.56.jpg type:images size:97545

+---------------------------------------+----------+---------+-------------+--------+--------+--------+---------+
| name                                  |   height |   width |   greyscale |      R |      G |      B |   ALPHA |
+=======================================+==========+=========+=============+========+========+========+=========+
| bw                                    |      512 |     512 |       21.48 |   0    |   0    |   0    |    0    |
+---------------------------------------+----------+---------+-------------+--------+--------+--------+---------+
| ciao                                  |     2048 |    1152 |        0    | 130.22 | 130.29 | 120.25 |    0    |
+---------------------------------------+----------+---------+-------------+--------+--------+--------+---------+
| daffodil                              |      500 |     335 |        0    | 109.23 |  85.52 |   4.77 |    0    |
+---------------------